<a href="https://colab.research.google.com/github/AUT-Student/NN-HW5/blob/main/NN_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: Neural Network
<br>
<b>Description:</b> HomeWork 5
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import prettytable
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Input, RNN, Average
import keras.backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# Dataset

In [ ]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data

Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
To: /content/lp1.data
100% 27.3k/27.3k [00:00<00:00, 171kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
To: /content/lp2.data
100% 14.6k/14.6k [00:00<00:00, 91.6kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
To: /content/lp3.data
100% 14.5k/14.5k [00:00<00:00, 90.9kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
To: /content/lp4.data
100% 33.6k/33.6k [00:00<00:00, 206kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data
To: /content/lp5.data
100% 49.2k/49.2k [00:00<00:00, 152kB/s]


In [ ]:
normal_label_datasets = ["normal","normal","ok","normal","normal"]

dataset_x = []
dataset_y = []
for i in range(5):
  normal_label = normal_label_datasets[i]
  dataset_i = pd.read_csv(f"/content/lp{i+1}.data", header=None)
  
  new_label = None
  new_data = None

  for j, row in dataset_i.iterrows():
    if j%16==0:
      new_data = []
      
      if row[0] == normal_label:
         new_label = 0
      else:
         new_label = 1

    else:
      new_data.append([float(item) for item in row[0].split()])

      if j%16==15:
        dataset_x.append(new_data)
        dataset_y.append(new_label)

In [ ]:
dataset_x, dataset_y = shuffle(dataset_x, dataset_y, random_state=0)

In [ ]:
dataset_len = len(dataset_x)

In [ ]:
train_x = dataset_x[:int(0.7*dataset_len)]
train_y = dataset_y[:int(0.7*dataset_len)]

valid_x = dataset_x[int(0.7*dataset_len):int(0.8*dataset_len)]
valid_y = dataset_y[int(0.7*dataset_len):int(0.8*dataset_len)]

test_x = dataset_x[int(0.8*dataset_len):]
test_y = dataset_y[int(0.8*dataset_len):]

In [ ]:
sum(train_y) * 100 / len(train_y)

71.60493827160494

In [ ]:
sum(test_y) * 100 / len(test_y)

77.41935483870968

# Network Model

## Elman

In [ ]:
class ElmanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/simple_rnn.py#L242-L492

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = units

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer="glorot_uniform", name="kernel")
      self.recurrent_kernel = self.add_weight(shape=(self.units, self.units), initializer="orthogonal", name="recurrent_kernel")
      self.bias = self.add_weight(shape=(self.units), initializer="zeros", name="bias")
      self.built = True

  def call(self, inputs, states):
      previous_hidden = states[0]
      current_hidden = K.dot(inputs, self.kernel) + K.dot(previous_hidden, self.recurrent_kernel)
      current_hidden = K.bias_add(current_hidden, self.bias)
      current_hidden = K.sigmoid(current_hidden)
      return current_hidden, [current_hidden]

In [ ]:
class ElmanNetwork(keras.Model):
  def __init__(self, cell_number):
    super().__init__()
    self.model = Sequential([
                             Input((15,6,)),
                             RNN(ElmanCell(cell_number)),
                             Dense(2, activation="softmax")
                            ])
  def call(self, inputs):
    return self.model.call(inputs)

## Jordan

In [ ]:
class JordanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/simple_rnn.py#L242-L492

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = 2

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer="glorot_uniform", name="kernel")
      self.recurrent_kernel = self.add_weight(shape=(2, self.units), initializer="orthogonal", name="recurrent_kernel")
      self.bias = self.add_weight(shape=(self.units), initializer="zeros", name="bias")
      
      self.output_kernel = self.add_weight(shape=(self.units, 2), name="output_kernel")
      self.output_bias = self.add_weight(shape=(2,), initializer="zeros", name="output_bias")

      self.built = True

  def call(self, inputs, states):
      previous_output = states[0]
      current_hidden = K.dot(inputs, self.kernel) + K.dot(previous_output, self.recurrent_kernel)
      current_hidden = K.bias_add(current_hidden, self.bias)
      current_hidden = K.sigmoid(current_hidden)

      current_output = K.dot(current_hidden, self.output_kernel)
      current_output = K.bias_add(current_output, self.output_bias)
      current_output = K.softmax(current_output)

      return current_output, [current_output]

In [ ]:
class JordanNetwork(keras.Model):
  def __init__(self, cell_number):
    super().__init__()
    self.model = Sequential([
                             Input((15,6,)),
                             RNN(JordanCell(cell_number)),
                            ])
  def call(self, inputs):
    return self.model.call(inputs)

# Ensemble Model

In [79]:
class EnsembleNetwork(keras.Model):
  def __init__(self, sub_models):
    super().__init__()
    
    self.sub_models = sub_models
    input = Input((15,6,))
    sub_output = [sub_model(input) for sub_model in sub_models]
    average = Average()
    output = average(sub_output)
    self.model = keras.Model(input, output)

  def number_sub_models(self):
    return len(self.sub_models)
    
  def sub_model(self, index):
    return self.sub_models[index]

  def call(self, inputs):
    return self.model.call(inputs)

# Single Network Results

## Elman

In [ ]:
table = prettytable.PrettyTable()
table.field_names = ["#Cell", "#Epoch", "Train Accuracy", "Validation Accuracy", "Test Accuracy"]

for cell_number in [8, 32, 64, 128, 512, 2048]:
  elman = ElmanNetwork(cell_number)

  es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

  elman.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  history = elman.fit(x=train_x, y=train_y, epochs=250, batch_size=16, validation_data=(valid_x, valid_y), callbacks=[es_callback], verbose=0)

  table.add_row([cell_number, history.epoch[-1]+1,
                 f"{round(elman.evaluate(x=train_x, y=train_y, verbose=0)[1]*100, 2)}%",
                 f"{round(elman.evaluate(x=valid_x, y=valid_y, verbose=0)[1]*100, 2)}%",
                 f"{round(elman.evaluate(x=test_x, y=test_y, verbose=0)[1]*100, 2)}%"
                 ])
print(table)

+-------+--------+----------------+---------------------+---------------+
| #Cell | #Epoch | Train Accuracy | Validation Accuracy | Test Accuracy |
+-------+--------+----------------+---------------------+---------------+
|   8   |  141   |     86.73%     |        80.43%       |     89.25%    |
|   32  |  193   |     97.84%     |        97.83%       |     95.7%     |
|   64  |   98   |     95.68%     |        89.13%       |     90.32%    |
|  128  |   45   |     93.83%     |        93.48%       |     90.32%    |
|  512  |   29   |     89.81%     |        82.61%       |     88.17%    |
|  2048 |   46   |     99.38%     |        95.65%       |     96.77%    |
+-------+--------+----------------+---------------------+---------------+


## Jordan

In [ ]:
table = prettytable.PrettyTable()
table.field_names = ["#Cell", "#Epoch", "Train Accuracy", "Validation Accuracy", "Test Accuracy"]

for cell_number in [8, 32, 64, 128, 512, 2048]:
  jordan = JordanNetwork(cell_number)

  es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

  jordan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  history = jordan.fit(x=train_x, y=train_y, epochs=250, batch_size=16, validation_data=(valid_x, valid_y), callbacks=[es_callback], verbose=0)

  table.add_row([cell_number, history.epoch[-1]+1,
                 f"{round(jordan.evaluate(x=train_x, y=train_y, verbose=0)[1]*100, 2)}%",
                 f"{round(jordan.evaluate(x=valid_x, y=valid_y, verbose=0)[1]*100, 2)}%",
                 f"{round(jordan.evaluate(x=test_x, y=test_y, verbose=0)[1]*100, 2)}%"
                 ])
print(table)

+-------+--------+----------------+---------------------+---------------+
| #Cell | #Epoch | Train Accuracy | Validation Accuracy | Test Accuracy |
+-------+--------+----------------+---------------------+---------------+
|   8   |   33   |     75.0%      |        76.09%       |     81.72%    |
|   32  |  157   |     95.37%     |        91.3%        |     94.62%    |
|   64  |   74   |     91.05%     |        73.91%       |     91.4%     |
|  128  |  118   |     97.22%     |        86.96%       |     89.25%    |
|  512  |   27   |     84.88%     |        65.22%       |     86.02%    |
|  2048 |   16   |     79.94%     |        76.09%       |     81.72%    |
+-------+--------+----------------+---------------------+---------------+


# Ensemble Network Results

## Single Dataset

In [88]:
def collect_results_single_dataset(ensemble_configs):
  table = prettytable.PrettyTable()
  table.field_names = ["Config", "Train Accuracy", "Validation Accuracy", "Test Accuracy"]

  for ensemble_name, ensemble_model in ensemble_configs:
    number_sub_models = ensemble_model.number_sub_models()

    for i in range(number_sub_models):
      es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
      ensemble_model.sub_model(i).compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
      history = ensemble_model.sub_model(i).fit(x=train_x, y=train_y,
                                                epochs=250, batch_size=16,
                                                validation_data=(valid_x, valid_y),
                                                callbacks=[es_callback], verbose=0)
    
    es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    ensemble_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                          loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    table.add_row([ensemble_name,
                  f"{round(ensemble_model.evaluate(x=train_x, y=train_y, verbose=0)[1]*100, 2)}%",
                  f"{round(ensemble_model.evaluate(x=valid_x, y=valid_y, verbose=0)[1]*100, 2)}%",
                  f"{round(ensemble_model.evaluate(x=test_x, y=test_y, verbose=0)[1]*100, 2)}%"
                  ])
  print(table)

### Only Elman

In [89]:
ensemble_configs = [
                    ("5 * Elman(8)", EnsembleNetwork([ElmanNetwork(8), ElmanNetwork(8), ElmanNetwork(8), ElmanNetwork(8), ElmanNetwork(8)])),
                    ("5 * Elman(32)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32)])),
                    ("5 * Elman(256)", EnsembleNetwork([ElmanNetwork(256), ElmanNetwork(256), ElmanNetwork(256), ElmanNetwork(256), ElmanNetwork(256)])),
                    ("5 * Elman(2048)", EnsembleNetwork([ElmanNetwork(2048), ElmanNetwork(2048), ElmanNetwork(2048), ElmanNetwork(2048), ElmanNetwork(2048)])),
                    ("Elman(32, 64, 128, 512, 2048)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(64), ElmanNetwork(128), ElmanNetwork(512), ElmanNetwork(2048)])),
                    ("Elman(32), Elman(2048)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(2048)]))
]

In [90]:
collect_results_single_dataset(ensemble_configs)

+-------------------------------+----------------+---------------------+---------------+
|             Config            | Train Accuracy | Validation Accuracy | Test Accuracy |
+-------------------------------+----------------+---------------------+---------------+
|          5 * Elman(8)         |     88.58%     |        71.74%       |     84.95%    |
|         5 * Elman(32)         |     98.77%     |        97.83%       |     96.77%    |
|         5 * Elman(256)        |     98.77%     |        97.83%       |     95.7%     |
|        5 * Elman(2048)        |     99.69%     |        100.0%       |     97.85%    |
| Elman(32, 64, 128, 512, 2048) |     98.46%     |        97.83%       |     94.62%    |
|     Elman(32), Elman(2048)    |     100.0%     |        100.0%       |     100.0%    |
+-------------------------------+----------------+---------------------+---------------+


### Only Jordan

In [91]:
ensemble_configs = [
                    ("5 * Jordan(8)", EnsembleNetwork([JordanNetwork(8), JordanNetwork(8), JordanNetwork(8), JordanNetwork(8), JordanNetwork(8)])),
                    ("5 * Jordan(32)", EnsembleNetwork([JordanNetwork(32), JordanNetwork(32), JordanNetwork(32), JordanNetwork(32), JordanNetwork(32)])),
                    ("5 * Jordan(128)", EnsembleNetwork([JordanNetwork(128), JordanNetwork(128), JordanNetwork(128), JordanNetwork(128), JordanNetwork(128)])),
                    ("5 * Jordan(256)", EnsembleNetwork([JordanNetwork(256), JordanNetwork(256), JordanNetwork(256), JordanNetwork(256), JordanNetwork(256)])),
                    ("Jordan(8, 32, 128, 256, 512)", EnsembleNetwork([JordanNetwork(8), JordanNetwork(32), JordanNetwork(128), JordanNetwork(256), JordanNetwork(512)])),
                    ("Jordan(32), Jordan(64)", EnsembleNetwork([JordanNetwork(32), JordanNetwork(64)]))
]

In [92]:
collect_results_single_dataset(ensemble_configs)

+------------------------------+----------------+---------------------+---------------+
|            Config            | Train Accuracy | Validation Accuracy | Test Accuracy |
+------------------------------+----------------+---------------------+---------------+
|        5 * Jordan(8)         |     84.57%     |        71.74%       |     87.1%     |
|        5 * Jordan(32)        |     91.05%     |        76.09%       |     92.47%    |
|       5 * Jordan(128)        |     96.6%      |        91.3%        |     96.77%    |
|       5 * Jordan(256)        |     93.83%     |        84.78%       |     88.17%    |
| Jordan(8, 32, 128, 256, 512) |     97.22%     |        84.78%       |     94.62%    |
|    Jordan(32), Jordan(64)    |     91.67%     |        80.43%       |     91.4%     |
+------------------------------+----------------+---------------------+---------------+


### Elman & Jordan

In [93]:
ensemble_configs = [
                    ("Elman(32, 2048), 3 * Jordan(128)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(2048), JordanNetwork(128), JordanNetwork(128), JordanNetwork(128)])),
                    ("3 * Elman(32), 2 * Jordan(32)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), JordanNetwork(32), JordanNetwork(32)])),
                    ("Elman(2048), Jordan(32)", EnsembleNetwork([ElmanNetwork(2048), JordanNetwork(32)])),
                    ("Elman(32, 2048), Jordan(32)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(2048), JordanNetwork(32)])),
                    ("Jordan(32, 64), Elman(128, 512, 2048)", EnsembleNetwork([JordanNetwork(32), JordanNetwork(64), ElmanNetwork(128), ElmanNetwork(512), ElmanNetwork(2048)])),
]

In [94]:
collect_results_single_dataset(ensemble_configs)

+---------------------------------------+----------------+---------------------+---------------+
|                 Config                | Train Accuracy | Validation Accuracy | Test Accuracy |
+---------------------------------------+----------------+---------------------+---------------+
|    Elman(32, 2048), 3 * Jordan(128)   |     95.99%     |        89.13%       |     90.32%    |
|     3 * Elman(32), 2 * Jordan(32)     |     100.0%     |        100.0%       |     98.92%    |
|        Elman(2048), Jordan(32)        |     98.15%     |        100.0%       |     93.55%    |
|      Elman(32, 2048), Jordan(32)      |     97.84%     |        97.83%       |     94.62%    |
| Jordan(32, 64), Elman(128, 512, 2048) |     95.68%     |        95.65%       |     92.47%    |
+---------------------------------------+----------------+---------------------+---------------+


## Multiple Dataset

In [102]:
def collect_results_multiple_dataset(ensemble_configs):
  table = prettytable.PrettyTable()
  table.field_names = ["Config", "Train Accuracy", "Validation Accuracy", "Test Accuracy"]

  for ensemble_name, ensemble_model in ensemble_configs:
    number_sub_models = ensemble_model.number_sub_models()

    for i in range(number_sub_models):
      es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
      ensemble_model.sub_model(i).compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
      
      len_train = len(train_y)
      sub_train_x = train_x[int(len_train*i/number_sub_models):int(len_train*(i+1)/number_sub_models)]
      sub_train_y = train_y[int(len_train*i/number_sub_models):int(len_train*(i+1)/number_sub_models)]
      
      history = ensemble_model.sub_model(i).fit(x=sub_train_x, y=sub_train_y,
                                                epochs=250, batch_size=16,
                                                validation_data=(valid_x, valid_y),
                                                callbacks=[es_callback], verbose=0)
    
    ensemble_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                          loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    table.add_row([ensemble_name,
                  f"{round(ensemble_model.evaluate(x=train_x, y=train_y, verbose=0)[1]*100, 2)}%",
                  f"{round(ensemble_model.evaluate(x=valid_x, y=valid_y, verbose=0)[1]*100, 2)}%",
                  f"{round(ensemble_model.evaluate(x=test_x, y=test_y, verbose=0)[1]*100, 2)}%"
                  ])
  print(table)

### Previous Best Settings

In [103]:
ensemble_configs = [
                    ("3 * Elman(32), 2 * Jordan(32)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), JordanNetwork(32), JordanNetwork(32)])),
                    ("Elman(2048), Jordan(32)", EnsembleNetwork([ElmanNetwork(2048), JordanNetwork(32)])),
                    ("5 * Jordan(128)", EnsembleNetwork([JordanNetwork(128), JordanNetwork(128), JordanNetwork(128), JordanNetwork(128), JordanNetwork(128)])),
                    ("Elman(32), Elman(2048)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(2048)]))
]

In [ ]:
collect_results_multiple_dataset(ensemble_configs)

+-------------------------------+----------------+---------------------+---------------+
|             Config            | Train Accuracy | Validation Accuracy | Test Accuracy |
+-------------------------------+----------------+---------------------+---------------+
| 3 * Elman(32), 2 * Jordan(32) |     81.17%     |        67.39%       |     88.17%    |
|    Elman(2048), Jordan(32)    |     89.51%     |        82.61%       |     90.32%    |
|        5 * Jordan(128)        |     79.63%     |        67.39%       |     74.19%    |
|     Elman(32), Elman(2048)    |     84.26%     |        67.39%       |     87.1%     |
+-------------------------------+----------------+---------------------+---------------+


### Incresing Subnetworks

In [108]:
ensemble_configs = [
                    ("2 * Elman(32)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(32)])),
                    ("4 * Elman(32)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32)])),
                    ("8 * Elman(32)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32),
                                                       ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32)])),
                    ("16 * Elman(32)", EnsembleNetwork([ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32),
                                                        ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32),
                                                        ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32),
                                                        ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32), ElmanNetwork(32)]))
]

In [109]:
collect_results_multiple_dataset(ensemble_configs)

+----------------+----------------+---------------------+---------------+
|     Config     | Train Accuracy | Validation Accuracy | Test Accuracy |
+----------------+----------------+---------------------+---------------+
| 2 * Elman(32)  |     86.42%     |        78.26%       |     88.17%    |
| 4 * Elman(32)  |     80.86%     |        67.39%       |     79.57%    |
| 8 * Elman(32)  |     78.7%      |        67.39%       |     86.02%    |
| 16 * Elman(32) |     79.32%     |        71.74%       |     80.65%    |
+----------------+----------------+---------------------+---------------+
